In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import numpy as np

# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")

city_data_df['Any Rain'] = ""
city_data_df['Any Rain'] = city_data_df['Rain inches (last 3hrs)'].apply(lambda x: 'no' if x == 0 else 'yes')

city_data_df['Any Snow'] = ""
city_data_df['Any Snow'] = city_data_df['Snow inches (last 3hrs)'].apply(lambda x: 'no' if x == 0 else 'yes')


city_data_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3hrs),Snow inches (last 3hrs),Any Rain,Any Snow
0,0,Nyurba,RU,2020-02-19 00:13:45,63.28,118.33,-14.28,97,35,4.74,scattered clouds,0,0,no,no
1,1,New Norfolk,AU,2020-02-19 00:13:45,-42.78,147.06,57.20,54,75,23.04,broken clouds,0,0,no,no
2,2,Qaanaaq,GL,2020-02-19 00:13:45,77.48,-69.36,-11.70,62,83,7.14,broken clouds,0,0,no,no
3,3,Kodiak,US,2020-02-19 00:13:45,57.79,-152.41,35.60,55,1,11.41,clear sky,0,0,no,no
4,4,Lubango,AO,2020-02-19 00:13:45,-14.92,13.49,61.59,84,100,3.60,overcast clouds,0,0,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554,554,Vila,PT,2020-02-19 00:15:11,42.03,-8.16,51.01,81,0,5.82,clear sky,0,0,no,no
555,555,Blagodatnoye,RU,2020-02-19 00:15:16,45.40,42.59,33.80,64,0,4.47,clear sky,0,0,no,no
556,556,Lar,IN,2020-02-19 00:15:16,26.20,83.97,52.34,57,0,3.76,clear sky,0,0,no,no
557,557,San Cristobal,VE,2020-02-19 00:15:16,7.77,-72.22,86.00,55,20,9.17,few clouds,0,0,no,no


In [3]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

# Ask the customer about the raining and snowing preferences
rain_pref = input("Do you want it to be raining? (yes/no) ")
snow_pref = input("Do you want it to be snowing? (yes/no) ")


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [4]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Any Rain"] == rain_pref) & \
                                       (city_data_df["Any Snow"] == snow_pref) ]
if len(preferred_cities_df.index) == 0:
    print("Sorry, but we didn't find a city with your preferred criteria. Please stop the kernel and re-run the whole notebook!")
else:
    print(f"Lucky you! we found {len(preferred_cities_df.index)} with your preferred criteria.")

Lucky you! we found 181 with your preferred criteria.


In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Arraial Do Cabo,BR,78.80,scattered clouds,-22.97,-42.03,
7,Rikitea,PF,79.20,clear sky,-23.12,-134.97,
10,Mahebourg,MU,78.80,scattered clouds,-20.41,57.70,
12,Puerto Ayora,EC,87.80,few clouds,-0.74,-90.35,
15,Georgetown,MY,77.00,few clouds,5.41,100.34,
20,Avarua,CK,80.60,light rain,-21.21,-159.78,
23,Maceio,BR,78.80,few clouds,-9.67,-35.74,
28,Lae,PG,76.69,broken clouds,-6.72,146.98,
31,Bambous Virieux,MU,78.80,scattered clouds,-20.34,57.76,
35,Luwuk,ID,82.94,overcast clouds,-0.95,122.79,


In [7]:
# Set parameters to search for a hotel.
params = {
    "radius" : 5000,
    "type" : "lodging",
    "key" : g_key
}

In [8]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# Create template for markers
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [10]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [11]:
# Add a marker layer for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")